## 使用Sarsa(Lambda)实现迷宫游戏
Sarsa 是一种单步更新法, 在环境中每走一步, 更新一次自己的行为准则, 我们可以在这样的 Sarsa 后面打一个括号, 说他是 Sarsa(0), 因为他等走完这一步以后直接更新行为准则. 如果延续这种想法, 走完这步, 再走一步, 然后再更新, 我们可以叫他 Sarsa(1). 同理, 如果等待回合完毕我们一次性再更新呢, 比如这回合我们走了 n 步, 那我们就叫 Sarsa(n). 为了统一这样的流程, 我们就有了一个 lambda 值来代替我们想要选择的步数, 这也就是 Sarsa(lambda) 的由来。

Lambda取值在0到1之间，当 lambda 取0, 就变成了 Sarsa 的单步更新, 当 lambda 取 1, 就变成了回合更新, 对所有步更新的力度都是一样. 当 lambda 在 0 和 1 之间, 取值越大, 离宝藏越近的步更新力度越大. 这样我们就不用受限于单步更新的每次只能更新最近的一步, 我们可以更有效率的更新所有相关步了。

In [1]:
from maze_env import Maze
from RL_brain import SarsaLambdaTable


def update():
    for episode in range(100):
        # initial observation
        observation = env.reset()

        # RL choose action based on observation
        action = RL.choose_action(str(observation))

        # initial all zero eligibility trace
        RL.eligibility_trace *= 0

        while True:
            # fresh env
            env.render()

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            # RL choose action based on next observation
            action_ = RL.choose_action(str(observation_))

            # RL learn from this transition (s, a, r, s, a) ==> Sarsa
            RL.learn(str(observation), action, reward, str(observation_), action_)

            # swap observation and action
            observation = observation_
            action = action_

            # break while loop when end of this episode
            if done:
                break

    # end of game
    print('game over')
    env.destroy()

if __name__ == "__main__":
    env = Maze()
    RL = SarsaLambdaTable(actions=list(range(env.n_actions)))

    env.after(100, update)
    env.mainloop()

game over


初始化 $Q(s,a)$，对于所有 $s\in\mathcal{S}$，$a\in\mathcal{A}(s)$  
循环（对于每个episode）
>初始化 Eligibility trace。 $E(s,a)=0$，对于所有 $s\in\mathcal{S}$，$a\in\mathcal{A}(s)$  
>初始化$S,A$  
>循环（对于episode中的每一步）
>>采取动作$A$，观察$R,S'$  
>>根据$S'$，以及查询$Q$，选择$A'$  
>>$\delta \leftarrow R + \gamma Q(S',A') - Q(S,A)$  
>>$E(S,A)\leftarrow E(S,A)+1$  
>> 对于所有 $s\in\mathcal{S}$，$a\in\mathcal{A}(s)$  
>>>$Q(s,a)\leftarrow Q(s,a)+\alpha\delta E(s,a)$  
>>>$E(s,a)\leftarrow \gamma \lambda E(s,a)$  

>>$S \leftarrow S',A\leftarrow A'$  

>直到S为终点